In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# Prepare the dataset with normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download and Load the training data
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle= False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 52.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.72MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.5MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.58MB/s]


In [3]:
# Build a simple feedforward neural network
class SimpleNet(nn.Module):
    def __init__(self):
       super(SimpleNet, self).__init__()
       self.fc1 = nn.Linear(28*28, 128)
       self.relu = nn.ReLU()
       self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
      x = x.view(-1, 28 * 28) # Flatten the image
      x = self.fc1(x)
      x = self.relu(x)
      x = self.fc2(x)
      return x

In [4]:
# create model instance
model = SimpleNet()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [5]:
# Train the model for 5 epochs
for epoch in range(5):
    running_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")


Epoch 1, Loss: 0.7540
Epoch 2, Loss: 0.3659
Epoch 3, Loss: 0.3214
Epoch 4, Loss: 0.2966
Epoch 5, Loss: 0.2763


In [12]:
from sklearn.metrics import classification_report
import torch.nn.functional as F
import numpy as np

# Evaluate and print classification report
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
  for images, labels in test_loader:
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
from sklearn.metrics import classification_report
import torch.nn.functional as F
import numpy as np

# Evaluate and print classification report
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
  for images, labels in test_loader:
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      all_preds.extend(predicted.numpy())
      all_labels.extend(labels.numpy())
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       980
           1       0.95      0.98      0.97      1135
           2       0.95      0.88      0.91      1032
           3       0.90      0.92      0.91      1010
           4       0.89      0.96      0.92       982
           5       0.94      0.86      0.89       892
           6       0.91      0.96      0.94       958
           7       0.93      0.93      0.93      1028
           8       0.90      0.88      0.89       974
           9       0.94      0.89      0.91      1009

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.93      0.92      0.92     10000



In [11]:
# save model
torch.save(model.state_dict(), "mnist_model.pth")
print("Model saved as mnist_model.pth")

Model saved as mnist_model.pth


In [13]:
# Plot training and validation accuracy & Loss

import matplotlib.pyplot as plt

# Initialize lists to store metrics during training
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Train the model for 5 epochs
for epoch in range(5):
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_losses.append(running_loss / len(train_loader))
    train_accuracies.append(100 * correct / total)

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

    # You'll need to add validation loop and calculate val_losses, val_accuracies here

# Handle unequal lengths or missing data gracefully
num_epochs = min(len(train_losses), len(val_losses), len(train_accuracies))

# Now you can proceed with plotting using train_losses, val_losses, train_accuracies, val_accuracies# Load model
model = SimpleNet()
model.load_state_dict(torch.load("mnist_model.pth"))
model.eval()
print("Model loaded and ready for inference")

Epoch 1, Loss: 0.2656, Accuracy: 92.27%
Epoch 2, Loss: 0.2657, Accuracy: 92.27%
Epoch 3, Loss: 0.2656, Accuracy: 92.27%
Epoch 4, Loss: 0.2655, Accuracy: 92.27%
Epoch 5, Loss: 0.2656, Accuracy: 92.27%
Model loaded and ready for inference
